In [1]:
from __future__ import absolute_import
from __future__ import print_function
import os
import glob
import random
import numpy as np
from keras import optimizers
from keras.layers import LSTM
from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16
from keras.layers.wrappers import TimeDistributed
from keras.applications.mobilenet import MobileNet
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.layers import Input
from keras.layers.core import Activation, Flatten, Reshape
from keras.layers.convolutional import Convolution2D, MaxPooling2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.applications import imagenet_utils

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
height = 120
width = 240
path = '../datasets/'
allfiles = glob.glob(path+'sensory/tile/*.npy')
sali = glob.glob(path+'content/saliencyImages/*.npy')
motion = glob.glob(path+'content/motionImages/*.npy')
prob = glob.glob(path+'sensory/tileProb/*.npy')

def myGenerator():
    while True:
        index_list = random.sample(range(1, 23999), 3)
        alldata_x = []
        alldata_y = []
        for i in index_list:
            f = allfiles[i]
            s = f.split('_')
            saliFile = '../datasets/content/saliencyImages/'+s[0][25:]+'_saliency_'+s[2].split('.')[0]+'.npy'
            motionFile = '../datasets/content/motionImages/'+s[0][25:]+'_motion_'+s[2].split('.')[0]+'.npy'
            probFile = '../datasets/sensory/tileProb/'+s[0][25:]+'_user'+s[1][4:]+'_'+s[2].split('.')[0]+'.npy'
            a = np.load(f)
            b = np.load(saliFile)
            c = np.load(motionFile)
            d = [a, b, c]
            alldata_x.append(d)
            alldata_y.append(np.load(probFile))
        alldata_x = np.array(alldata_x)
        alldata_x = np.rollaxis(alldata_x, 1, 5)  
        #alldata_x = alldata_x.reshape((32, 30, height, width, 3))
        #alldata_x = np.swapaxes(alldata_x, 1, 4)
        alldata_y = np.array(alldata_y)
        yield alldata_x, alldata_y
# x = myGenerator()
# xtrain, ytrain = next(x)
# print('xtrain shape:',xtrain.shape)
# print('ytrain shape:',ytrain.shape)

In [3]:
# load the numpy arrays from saliency, motion maps and sensor data
sali = glob.glob(path+'content/saliencyImages/*.npy')
motion = glob.glob(path+'content/motionImages/*.npy')
sensory = glob.glob(path+'sensory/tile/*.npy')
input_shape=(30, height, width, 3)

In [7]:
def mySegNet(input_shape):
    base_model  = MobileNet(input_shape=(224,224,3), include_top=False)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    cnn_model = Model(inputs=base_model.input, outputs=x)
    
    model = Sequential();
    model.add(TimeDistributed(cnn_model, input_shape=input_shape))
    model.add(TimeDistributed(Flatten()))
    
    model.add(LSTM(200, return_sequences=True))
    model.compile(optimizer='adam', loss='mean_squared_error')
    #print(model.summary())
    return model 

In [ ]:
input_shape=(30, height, width, 3)
model = mySegNet(input_shape)

model.fit_generator(generator=myGenerator(),
                    use_multiprocessing=True,
                   steps_per_epoch=300, epochs=100)
model.save('model1.h5')
model.save_weights('model_weights1.h5')


In [6]:
model.fit_generator(generator=myGenerator(),
                    use_multiprocessing=True,
                   steps_per_epoch=300, epochs=100)
model.save('model2.h5')
model.save_weights('model_weights2.h5')

Epoch 1/100
300/300 [==============================] - 168s 560ms/step - loss: 0.1071
Epoch 2/100
300/300 [==============================] - 167s 558ms/step - loss: 0.1117
Epoch 3/100
300/300 [==============================] - 168s 560ms/step - loss: 0.1134
Epoch 4/100
300/300 [==============================] - 167s 557ms/step - loss: 0.1057
Epoch 5/100
300/300 [==============================] - 168s 558ms/step - loss: 0.1036
Epoch 6/100
300/300 [==============================] - 169s 563ms/step - loss: 0.1098
Epoch 7/100
300/300 [==============================] - 167s 557ms/step - loss: 0.1048
Epoch 8/100
300/300 [==============================] - 168s 561ms/step - loss: 0.1061
Epoch 9/100
300/300 [==============================] - 166s 554ms/step - loss: 0.1042
Epoch 10/100
300/300 [==============================] - 168s 560ms/step - loss: 0.1074
Epoch 11/100
300/300 [==============================] - 169s 563ms/step - loss: 0.1091
Epoch 12/100
300/300 [==============================

In [7]:
model.fit_generator(generator=myGenerator(),
                    use_multiprocessing=True,
                   steps_per_epoch=300, epochs=100)
model.save('model3.h5')
model.save_weights('model_weights3.h5')

Epoch 1/100
300/300 [==============================] - 170s 566ms/step - loss: 0.0126
Epoch 2/100
300/300 [==============================] - 170s 565ms/step - loss: 0.0121
Epoch 3/100
300/300 [==============================] - 168s 559ms/step - loss: 0.0129
Epoch 4/100
300/300 [==============================] - 169s 563ms/step - loss: 0.0122
Epoch 5/100
300/300 [==============================] - 169s 564ms/step - loss: 0.0124
Epoch 6/100
300/300 [==============================] - 169s 564ms/step - loss: 0.0120
Epoch 7/100
300/300 [==============================] - 169s 565ms/step - loss: 0.0118
Epoch 8/100
300/300 [==============================] - 169s 564ms/step - loss: 0.0118
Epoch 9/100
300/300 [==============================] - 169s 564ms/step - loss: 0.0127
Epoch 10/100
300/300 [==============================] - 169s 564ms/step - loss: 0.0116
Epoch 11/100
300/300 [==============================] - 168s 561ms/step - loss: 0.0116
Epoch 12/100
300/300 [==============================

In [10]:
count = 0
batchSize = 3
allfiles = glob.glob('../datasets/testData/sens/*.npy')
def myTestDataGenerator():
    global count
    while True:
        index_list = range(count, count+batchSize)
        count += batchSize
        alldata_x = []
        alldata_y = []
        for i in index_list:
            f = allfiles[i]
            s = f.split('_')
            saliFile = '../datasets/content/saliencyImages/'+s[0][25:]+'_saliency_'+s[2].split('.')[0]+'.npy'
            motionFile = '../datasets/content/motionImages/'+s[0][25:]+'_motion_'+s[2].split('.')[0]+'.npy'
            probFile = '../datasets/sensory/tileProb/'+s[0][25:]+'_user'+s[1][4:]+'_'+s[2].split('.')[0]+'.npy'
            a = np.load(f)
            b = np.load(saliFile)
            c = np.load(motionFile)
            d = [a, b, c]
            alldata_x.append(d)
            alldata_y.append(np.load(probFile))
        alldata_x = np.array(alldata_x)
        alldata_x = np.rollaxis(alldata_x, 1, 5)  
        #alldata_x = alldata_x.reshape((32, 30, height, width, 3))
        #alldata_x = np.swapaxes(alldata_x, 1, 4)
        alldata_y = np.array(alldata_y)
        yield alldata_x, alldata_y
# x = myTestDataGenerator()
# xtest, ytest = next(x)
# print('xtrain shape:',xtest.shape)
# print('ytrain shape:',ytest.shape)

In [11]:
input_shape=(30, height, width, 3)
model = mySegNet(input_shape)
model.load_weights('model_weights2.h5')
totalTestSamples = len(allfiles)
predictions = []
ytrue = []
for i in range(0, totalTestSamples, batchSize):
    x = myTestDataGenerator()
    xtest, ytest = next(x)
    ytrue.append(ytest)
    pred = model.predict(xtest, batch_size=batchSize)
    for p in pred:
        predictions.append(p)
print('predictions shape: ', np.array(predictions).shape)

predictions shape:  (6000, 30, 200)


In [20]:
Y = []
for i in ytrue:
    for j in i:
        Y.append(j)
Y = np.array(Y)
Y.shape

(6000, 30, 200)

In [24]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
thresh = 0
sampleSize = 30
avgPrec = []
avgAcc = []
avgRec = []
for sample in range(totalTestSamples):
    prec = []
    acc = []
    rec = []
    for frame in range(sampleSize):
        temp = predictions[sample][frame]
        temp[temp > thresh] = 1
        temp[temp <= thresh] = 0
        yt = Y[sample][frame]
        yp = temp
        #print(np.array(yt).shape, np.array(yp).shape)
        prec.append(precision_score(yt, yp, average="macro"))
        acc.append(accuracy_score(yt, yp)) 
        rec.append(recall_score(yt, yp, average="macro"))
    avgPrec.append(np.mean(prec))
    avgAcc.append(np.mean(acc))
    avgRec.append(np.mean(rec))
    print('Precision: '+str(np.mean(prec)), 'Accuracy: '+str(np.mean(acc)), 'Recall: '+str(np.mean(rec)))
print('Average Precision: ', np.mean(avgPrec))
print('Average Accuracy: ', np.mean(avgAcc))
print('Average Recall: ', np.mean(avgRec))

Precision: 0.9060723964382501 Accuracy: 0.962 Recall: 0.9773053004719191
Precision: 0.898502253911465 Accuracy: 0.9593333333333333 Recall: 0.975447065799653
Precision: 0.9133721862728831 Accuracy: 0.9638333333333332 Recall: 0.9782063146434403
Precision: 0.9353798728477805 Accuracy: 0.9755000000000001 Recall: 0.9853485993381204
Precision: 0.9088397534157767 Accuracy: 0.9656666666666666 Recall: 0.9793280300848323
Precision: 0.9124447956901313 Accuracy: 0.9626666666666666 Recall: 0.9770847119213981
Precision: 0.8904200340863035 Accuracy: 0.9526666666666664 Recall: 0.9716019967428148
Precision: 0.9285036925367401 Accuracy: 0.9733333333333333 Recall: 0.9842164237401252
Precision: 0.9433048433048429 Accuracy: 0.9795000000000001 Recall: 0.987797619047619
Precision: 0.9242791121430678 Accuracy: 0.9696666666666668 Recall: 0.9817760440252594
Precision: 0.9301389093649158 Accuracy: 0.9750000000000002 Recall: 0.985231465367212
Precision: 0.908931131253653 Accuracy: 0.9660000000000003 Recall: 0.979

Precision: 0.9264410576266682 Accuracy: 0.9715000000000004 Recall: 0.9829786854861702
Precision: 0.9402564310138121 Accuracy: 0.9781666666666669 Recall: 0.9870039682539681
Precision: 0.9112344270239008 Accuracy: 0.9666666666666665 Recall: 0.9803166063823362
Precision: 0.9271890728173293 Accuracy: 0.9734999999999999 Recall: 0.9843495664122599
Precision: 0.8966507426391893 Accuracy: 0.9584999999999998 Recall: 0.9753695996285816
Precision: 0.9103776915554839 Accuracy: 0.9666666666666666 Recall: 0.980340526743242
Precision: 0.9186993417256575 Accuracy: 0.9693333333333332 Recall: 0.9818375536484018
Precision: 0.9046518135788446 Accuracy: 0.9606666666666666 Recall: 0.9764556658341376
Precision: 0.9067274038738751 Accuracy: 0.9618333333333333 Recall: 0.9771366904744727
Precision: 0.8954646065496793 Accuracy: 0.9583333333333334 Recall: 0.9752999624016533
Precision: 0.9231782617425035 Accuracy: 0.9720000000000001 Recall: 0.9834840662103105
Precision: 0.9034509840109967 Accuracy: 0.9606666666666

Precision: 0.8555905507395334 Accuracy: 0.9361666666666666 Recall: 0.9622757860540664
Precision: 0.9667499601466604 Accuracy: 0.9881666666666663 Recall: 0.9929502751330249
Precision: 0.9433221732644071 Accuracy: 0.9788333333333334 Recall: 0.9873253493013971
Precision: 0.9186033222792804 Accuracy: 0.9683333333333334 Recall: 0.9811355914886608
Precision: 0.9230571675372704 Accuracy: 0.9710000000000001 Recall: 0.9824020198481428
Precision: 0.8866505826454479 Accuracy: 0.9544999999999997 Recall: 0.9731210117182966
Precision: 0.9219489359846503 Accuracy: 0.9608333333333333 Recall: 0.9752652223814454
Precision: 0.9174475665722807 Accuracy: 0.968333333333333 Recall: 0.9811765030097597
Precision: 0.9187027583417184 Accuracy: 0.9688333333333333 Recall: 0.9815127500704423
Precision: 0.9068095769091009 Accuracy: 0.9628333333333334 Recall: 0.9778878839936289
Precision: 0.8569997364018145 Accuracy: 0.9358333333333333 Recall: 0.9618420000782087
Precision: 0.9161747808586701 Accuracy: 0.9681666666666

Precision: 0.9258956628302042 Accuracy: 0.9658333333333334 Recall: 0.9782223330879524
Precision: 0.9354540908215387 Accuracy: 0.9653333333333332 Recall: 0.97680361421903
Precision: 0.913060714087672 Accuracy: 0.9648333333333332 Recall: 0.9789267465954752
Precision: 0.9419457976992466 Accuracy: 0.9776666666666664 Recall: 0.9858521392129236
Precision: 0.9321266970288155 Accuracy: 0.9743333333333337 Recall: 0.9846787581500844
Precision: 0.8786840711075471 Accuracy: 0.949 Recall: 0.969810589362566
Precision: 0.8486120114748059 Accuracy: 0.9295000000000001 Recall: 0.9581239502497667
Precision: 0.9417949930759529 Accuracy: 0.9788333333333331 Recall: 0.9874255815189106
Precision: 0.9188693957115012 Accuracy: 0.9688333333333331 Recall: 0.9814484126984129
Precision: 0.8822786935286935 Accuracy: 0.9516666666666665 Recall: 0.971417136224625
Precision: 0.8796492813399535 Accuracy: 0.9051666666666666 Recall: 0.9227139459392674
Precision: 0.9186830066017659 Accuracy: 0.9576666666666667 Recall: 0.969

Precision: 0.8818205301966611 Accuracy: 0.9486666666666668 Recall: 0.9693529062824762
Precision: 0.9161370820462681 Accuracy: 0.9653333333333334 Recall: 0.9791806565765876
Precision: 0.896960592575654 Accuracy: 0.9539999999999998 Recall: 0.9721003492205225
Precision: 0.8642171336001306 Accuracy: 0.9430000000000001 Recall: 0.9664128041610773
Precision: 0.9107014564536764 Accuracy: 0.964 Recall: 0.9785375622382887
Precision: 0.8870596780499646 Accuracy: 0.9410000000000002 Recall: 0.9633374345151655
Precision: 0.9213854981525352 Accuracy: 0.9706666666666667 Recall: 0.9826563406427659
Precision: 0.9307799952825626 Accuracy: 0.9731666666666668 Recall: 0.983953044444318
Precision: 0.8717162803611863 Accuracy: 0.9431666666666667 Recall: 0.9662642942944107
Precision: 0.9470469369811474 Accuracy: 0.9791666666666667 Recall: 0.9873755109128309
Precision: 0.9032662540221333 Accuracy: 0.9631666666666666 Recall: 0.9782701009397842
Precision: 0.9276780380988208 Accuracy: 0.9721666666666667 Recall: 0.

...

Precision: 0.8932143727217541 Accuracy: 0.9566666666666666 Recall: 0.9742695332570896
Precision: 0.9426233271086208 Accuracy: 0.9795 Recall: 0.9878381234150463
Precision: 0.9144393121560307 Accuracy: 0.9686666666666667 Recall: 0.9815732057969967
Precision: 0.9051533443920661 Accuracy: 0.9629999999999997 Recall: 0.9780694161293709
Precision: 0.9197386567969273 Accuracy: 0.9681666666666667 Recall: 0.9809492604005988
Precision: 0.8350007219237541 Accuracy: 0.9259999999999999 Recall: 0.95652201480613
Precision: 0.8868577622028584 Accuracy: 0.9531666666666664 Recall: 0.9721488447449983
Precision: 0.9174983489821114 Accuracy: 0.9671666666666668 Recall: 0.9803644217588915
Precision: 0.924143061319704 Accuracy: 0.9713333333333334 Recall: 0.9826056134978646
Precision: 0.9143916566239021 Accuracy: 0.9668333333333332 Recall: 0.9803163127857288
Precision: 0.9044147304102299 Accuracy: 0.9628333333333333 Recall: 0.9779969847899712
Precision: 0.8674966463323656 Accuracy: 0.9406666666666665 Recall: 0.

Precision: 0.941757450647053 Accuracy: 0.9781666666666667 Recall: 0.9869261477045908
Precision: 0.8365634274422431 Accuracy: 0.920666666666667 Recall: 0.9527777777777776
Precision: 0.9591619273972211 Accuracy: 0.985333333333333 Recall: 0.9912698412698416
Precision: 0.918200192540493 Accuracy: 0.9678333333333333 Recall: 0.9804305960788591
Precision: 0.927171096520942 Accuracy: 0.9733333333333333 Recall: 0.9842425595450126
Precision: 0.8649547042157608 Accuracy: 0.9396666666666668 Recall: 0.9640795683920972
Precision: 0.930065938623384 Accuracy: 0.9736666666666666 Recall: 0.984303295190578
Precision: 0.8745089856588114 Accuracy: 0.9451666666666667 Recall: 0.9672434892120518
Precision: 0.8876066410197331 Accuracy: 0.9533333333333333 Recall: 0.9723670849010214
Precision: 0.9289841096012875 Accuracy: 0.9644999999999999 Recall: 0.9774954503430724
Precision: 0.8273708572863023 Accuracy: 0.9120000000000003 Recall: 0.9467399924592717
Precision: 0.91686461765721 Accuracy: 0.965 Recall: 0.9787969

Precision: 0.9229947153631366 Accuracy: 0.9711666666666666 Recall: 0.9828807410538177
Precision: 0.8973672900845486 Accuracy: 0.9611666666666667 Recall: 0.9771683293200322
Precision: 0.9023013803488351 Accuracy: 0.9274999999999999 Recall: 0.9411783416439017
Precision: 0.8434197221490056 Accuracy: 0.9248333333333332 Recall: 0.9550624400549553
Precision: 0.9232546605259091 Accuracy: 0.9676666666666666 Recall: 0.9803460748646365
Precision: 0.9177767316008446 Accuracy: 0.9416666666666664 Recall: 0.9551265532835954
Precision: 0.9384402114665273 Accuracy: 0.976333333333333 Recall: 0.9858168704676394
Precision: 0.8622920283617093 Accuracy: 0.9356666666666668 Recall: 0.9613653664367063
Precision: 0.9257038768739287 Accuracy: 0.9726666666666669 Recall: 0.983810186796612
Precision: 0.8660598247387937 Accuracy: 0.9381666666666665 Recall: 0.9629843775652158
Precision: 0.9299549759843878 Accuracy: 0.9751666666666665 Recall: 0.9853884183522192
Precision: 0.8479770997884766 Accuracy: 0.93149999999999

Precision: 0.8725314624183335 Accuracy: 0.9464999999999999 Recall: 0.9683994662953938
Precision: 0.8962652168978071 Accuracy: 0.9558333333333334 Recall: 0.9737166606175452
Precision: 0.94687065371952 Accuracy: 0.9800000000000001 Recall: 0.9848934762059981
Precision: 0.8766240209654844 Accuracy: 0.9496666666666667 Recall: 0.9703080403759136
Precision: 0.8919963438425443 Accuracy: 0.9553333333333334 Recall: 0.9734492518203942
Precision: 0.9368032902654467 Accuracy: 0.9570000000000001 Recall: 0.9694473816896653
Precision: 0.9240725178960475 Accuracy: 0.9716666666666663 Recall: 0.9832143329405368
Precision: 0.7942794310010143 Accuracy: 0.8779999999999999 Recall: 0.9260596768237765
Precision: 0.8740052373305739 Accuracy: 0.9436666666666669 Recall: 0.9665666730202928
Precision: 0.9319907407407406 Accuracy: 0.9755 Recall: 0.9855074889916519
Precision: 0.9132359781870188 Accuracy: 0.9663333333333332 Recall: 0.9799725675445392
Precision: 0.8724647769922949 Accuracy: 0.9453333333333334 Recall: 0

Precision: 0.9007078534950846 Accuracy: 0.9421666666666668 Recall: 0.9574166366512522
Precision: 0.9433048433048429 Accuracy: 0.9795000000000001 Recall: 0.987797619047619
Precision: 0.9071843185630374 Accuracy: 0.9643333333333334 Recall: 0.9789150906911087
Precision: 0.8579925808612079 Accuracy: 0.9371666666666667 Recall: 0.9623819511481694
Precision: 0.9160466336782128 Accuracy: 0.9671666666666668 Recall: 0.9804260526565914
Precision: 0.9342672041548803 Accuracy: 0.9758333333333334 Recall: 0.9856605821473257
Precision: 0.9257086786034154 Accuracy: 0.9724999999999999 Recall: 0.9837036246900229
Precision: 0.9171158553511491 Accuracy: 0.9703333333333336 Recall: 0.9825816993464052
Precision: 0.9084495021337123 Accuracy: 0.9638333333333333 Recall: 0.9784722222222225
Precision: 0.8561961039595299 Accuracy: 0.9275000000000002 Recall: 0.9532006126968627
Precision: 0.852623492542325 Accuracy: 0.9321666666666667 Recall: 0.9596087586731302
Precision: 0.9336651794430432 Accuracy: 0.97599999999999

In [ ]:
index  = 28
thresh = 0.5

temp = predictions[0][index] 
temp[temp > thresh] = 1
temp[temp <= thresh] = 0

for i, j in enumerate(ytest[0][index]):
    if ytest[0][index][i] != temp[i]:
        print('Index: ', i, 'Value: ', ytest[0][index][i], temp[i])

In [ ]:
print(ytest[0][index].shape)